In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import scipy.linalg
import scipy.interpolate

# Interpolation

If we have sampled data at some fixed set of points, we often want to systematically estimate the value our data would take in between these sampled points. In other words, we want to come up with a systematic way to generate a value for the function we have sampled at a point we have not sampled directly.

There could be many reasons for this:
- You might have performed a number of expensive simulations where some parameter $r$ has been varied, and want a sensible estimate of some parameter you're deriving from your simulation at a value of $r$ you have not calculated directly.
- You might want to find the derivative or integral as a function of $r$.
- You just want a nice-looking plot.

The interpolating function should pass through all the sampled points, otherwise we are no longer interpolating, but curve fitting (which we'll cover later in the course).

Let's say we have obtained $f(x_i)$ for some set $x_i$, with $0\le i\le n$. This means we want to find some good approximation for $f(t)$ where $x_i < t < x_{i+1}$ for some $i$ and $i+1$ in our data set.

Lets generate some random data that we can use as an example:

In [ ]:
# Lets say our x-values are ordered, from 0 to 10 and y have random y values
# Generate 11 evenly spaced x-values
x_data = np.linspace(0, 10, 11)
# Generate 11 random y-values between 0 and 1
y_data = np.random.rand(11)
# To get something that looks more well behaved you could sort the
# y-values by uncommenting the following line.
#y_data.sort()

plt.plot(x_data, y_data, 'bo')
plt.show()

## Constant Interpolation

The simplest thing to do is to assume our data is constant in the interval, and equal to the value at one of the end-points.

For example we could take for $x_i < t < x_{i+1}$, $f(t) = f(x_i)$ or $f(t) = f(x_{i+1})$.

A python function which does this is below:

In [ ]:
def constant_interp(xvals, yvals, t):
    '''Interpolate the value of y(t) given y(x_i) using constant approx
    
    Parameters
    ----------
    xvals : float, array
        The x-values we have sampled - they must be in increasing order.
    yvals : float array
        The value of y at each x we have sampled.
    t : float
        The (x-axis) value at which we want to interpolate.
    
    Notes
    -----
    This will use the value on the right side of the interval.   
    '''
    
    # Here we use searchsorted from numpy which tells us where t would need to
    # be in our list of x-vals to maintain it as sorted. This corresponds to
    # the right hand index of the interval.
    t_index = np.searchsorted(xvals, t, 'left')
    
    return yvals[t_index]

Lets see how this looks if we plot it with the random input we generated earlier:

In [ ]:
xplt = np.linspace(0, 10, 1001)
# We just plot dots instead of lines here as an easy way to highlight discontinuities.
plt.plot(xplt, constant_interp(x_data, y_data, xplt), 'r.', x_data, y_data, 'bo')
plt.xlabel('x')
plt.ylabel('Constant Interpolated y')
plt.show()

This doesn't look great. But as you can imagine, with a sufficiently dense set of input data, may be adequate. It would be nice to have an interpolating function without discontinuities though. Here every value calculated for our interpolating function has used only a single point. You can likely guess the easiest improvement that can be made to this.

### _Task_

- Modify the constant interpolation routine to use return the constant value given by the left value of the interval rather than the right.
- A slightly better approach might be to take the function value from the nearest sampled x-value. How might you implement this?

## Linear Interpolation

The next step up would then be to use a linear interpolation, where the interpolating function is defined piecewise as the line connecting each point we wish to interpolate between.

Given two different points we can define the line that passes through them both as $$y(x) = \frac{y_2-y_1}{x_2-x_1}(x-x_1)+y_1,$$ or $$y(x) = \frac{y_2-y_1}{x_2-x_1}(x-x_2)+y_2.$$

You could equivalently think of this as having some normalized weighting on the interval, with the weights given by the relative proximity to the sampled x-values:
$$ y(x) = A(x)y_1 + B(x)y_2\\
A(x) = \frac{x_2 - x}{x_2 - x_1} \\
B(x) = \frac{x - x_1}{x_2 - x_1} \\
A(x) + B(x) = 1 $$

We can use this to define a python function as follows:

In [ ]:
def linear_interp(xvals, yvals, t):
    '''Linear interpolation of the value of y(t) given y(x_i)
    
    Parameters
    ----------
    xvals : float, array
        the x-values we have sampled - they must be in increasing order.
    yvals : float, array
        the value of y at each x we have sampled.
    t : float
        the (x-axis) value at which we want to interpolate.
    
    Returns
    -------
    y_interp : float
        the interpolated value of y(t).
    '''
    
    # Here we use searchsorted from numpy which tells us where t would need to
    # be in our list of x-vals to maintain it as sorted.
    t_index = np.searchsorted(xvals, t, 'left')
    
    y_interp = (yvals[t_index] + (t - xvals[t_index]) *
                (yvals[t_index] - yvals[t_index-1])/(xvals[t_index] - xvals[t_index-1]))
    
    return y_interp

Now lets see how this looks with our random data

In [ ]:
xplt = np.linspace(0, 10, 1001)
plt.plot(xplt, linear_interp(x_data, y_data, xplt), 'r-', x_data, y_data, 'bo')
plt.xlabel('x')
plt.ylabel('Linear Interpolated y')
plt.show()

Already this looks much better. The interpolating function is now continuous, however its derivative is not, so the interpolating function looks pretty jagged.

## Lagrange Polynomial

Let's change our approach a bit, and instead of trying to find some piecewise function, let's instead try to find a polynomial that will pass through all our sampled points. Polynomials are always well behaved, and can be differentiated as much as we want, so maybe this will give us a good interpolation. The polynomial of lowest degree that will pass through a fixed set of points is known as the Lagrange polynomial.

Say we have four points: $(0, 2)$, $(1, 1)$, $(2, 2)$, $(3,0)$, and we want to find the polynomial that will pass through them. The polynomial could be written as $a_0+a_1x+a_2x^2+a_3x^3=y$. Since the four points given must satisfy this polynmoial, we could write this as the following matrix equation:

$$ \begin{pmatrix} 1 & 0 & 0 & 0 \\ 1 & 1 & 1 & 1 \\ 1 & 2 & 4 & 8 \\ 1 & 3 & 9 & 27 \end{pmatrix} 
\begin{pmatrix}a_0 \\ a_1 \\ a_2 \\ a_3 \end{pmatrix} =
\begin{pmatrix}2 \\ 1 \\ 2 \\ 0 \end{pmatrix} $$

Here each row has values $x^i$ with $i$ the column index, with a row for each $x$ value. We know how to solve this type of system from the previous class. So let's do this now in python and see how it looks.

In [ ]:
points = np.array([[0, 2], [1, 1], [2, 2], [3, 0]])

xmat = np.empty((len(points), len(points)))
for i in range(len(points)):
    xmat[:, i] = points[:, 0] ** i

coeffs = scipy.linalg.solve(xmat, points[:, 1])
xplt = np.linspace(points[:, 0].min(), points[:,0].max(), 100)

# polyval will evaluate a polynomial given coefficients but
# expects them in the reverse order to that we calculated.
yplt = np.polyval(coeffs[::-1], xplt)

plt.plot(points[:, 0], points[:, 1], 'bo')
plt.plot(xplt, yplt, 'r-')
plt.show()

This looks pretty good in this case.

A fairly compact formula can be found for this polynomial that's a little easier than solving a matrix equation. We can write the interpolating polynomial as

$$ L(x) = \Sigma_{j=0}^k y_j l_j(x) $$
with
$$ l_j(x) = \Pi_{m=0, m\ne j}^{k} \frac{x-x_m}{x_j-x_m}$$

As we saw previously, solving a matrix equation is O($n^3$), but this expression should give us the interpolating polynomial in O($n^2$) operations. Let's write a python function to calculate this and see if we get the same thing.

In [ ]:
def lpoly_interp(xvals, yvals, t):
    '''
    Lagrange interpolating polynomial of the value of y(t) given y(x_i)
    
    Parameters
    ----------
    xvals : float, array
        the x-values we have sampled. These must all be different.
    yvals : float, array
        the value of y at each x we have sampled.
    t : float
        the (x-axis) value at which we want to interpolate.
    
    Returns
    -------
    y_interp : float
        the interpolated value of y(t).
    '''
    k = len(xvals)
    y_interp = 0
    for j in range(k):
        # It's important to specify the axis for np.prod or we won't be able to use the function
        # with an array argument.
        lj = np.prod([(t - xvals[m]) / (xvals[j] - xvals[m]) for m in range(k) if m != j], axis=0)
        y_interp += yvals[j] * lj
    return y_interp

In [ ]:
# Now let's see if we get the same result as before:
xplt = np.linspace(points[:, 0].min(), points[:,0].max(), 100)
yplt = lpoly_interp(points[:, 0], points[:, 1], xplt)

plt.plot(points[:, 0], points[:, 1], 'bo')
plt.plot(xplt, yplt, 'r-')
plt.show()

This looks identical to what we had before.

Let's also see how it looks with the random data we generated at the beginning.

In [ ]:
xplt = np.linspace(0, 10, 1001)
plt.plot(xplt, lpoly_interp(x_data, y_data, xplt), 'r-', x_data, y_data, 'bo')
plt.xlabel('x')
plt.ylabel('Lagrange Interpolated y')
plt.show()

Depending on how how your random data looks your interpolation may look very different. You'll likely see it goes very far outside the range of the original sampled y values in a number of places. This is known as [Runge's phenomenon](https://en.wikipedia.org/wiki/Runge%27s_phenomenon). Runge found that the if the function $f(x)=\frac{1}{1+25x^2}$ was interpolated from a set of equidistant samples, it showed large oscillations toward the edges of the sampled data.

It would be useful to have an approach that kept some of the smoothness of the polynomial interpolation, but without this kind of effect.

## Spline Interpolation

Let's return to the piecewise interpolating approach we used earlier, but try to improve the smoothness of the interpolation. A good approach to do this is to use what's known as a spline. A spline is a piecewise polynomial function. Let's think about what we'll need to generate it in such a way that the interpolating spline function and both it's first and second derivatives are continuous.

Let's look at the expression for a cubic polynomial $S(x)$ passing through a pair of points $(x_0, y_0)$ and $(x_1, y_1)$.

Since it passes through both points $S(x_0) = y_0$ and $S(x_1) = y_1$. Let's also denote $S'(x_0) = k_0$ and $S'(x_1) = k_1$. We can write a cubic polynomial in the following symmetric form

$$ S(x) = (1-t)y_0+ty_1 + t(1-t)[a(1-t)+bt] $$

with $ t=\frac{x-x_0}{\Delta x_0}$, $\Delta x_0 = x_1 - x_0$, $\Delta y_0 = y_1 -y_2$,  $a=k_0\Delta x_0- \Delta y_0$ and $b=-k_1\Delta x_0+\Delta y_0$.

Now we can find the first and second derivatives of this and show that

$$ S'(x) = \frac{\Delta y_0}{\Delta x_0} + (1-2t)\frac{a(1-t)+bt}{\Delta x_0} + t(1-t)\frac{b-a}{\Delta x_0} $$

and

$$ S''(x) = 2\frac{b-2a+3t(a-b)}{\Delta x_0^2} $$

At this point you can test that the definiations we gave for $a$ and $b$ above will give us $S'(x_0)=k_0$ and $S'(x_1)=k_1$. And we can also find that

$$ S''(x_0) = 2\frac{b-2a}{\Delta x_0^2} $$

and

$$ S''(x_1) = 2\frac{a-2b}{\Delta x_0^2} $$

Now let's say we have some set of $n+1$ points $(x_i, y_i)$ with $0 \le i \le n$, such that we have $n$ intervals. For each interval we can define the cubic polynomial passing through the points bounding the interval as

$$ S_i = (1-t)y_i + ty_{i+1} + t(1-t)[a_i(1-t)+b_i t] $$

for $0 \le i \le n-1$, with $t=\frac{x-x_i}{\Delta x_i}$, $\Delta x_i = x_{i+1} - x_i$, $\Delta y_i = y_{i+1} - y_i$, $a_i=k_i\Delta x_i-\Delta y_i$ and $b_i=-k_{i+1}\Delta x_i + \Delta y_i$.

If we want our interpolation to be differentiable, we need $S'_i(x_{i+1})=S'_{i+1}(x_{i+1})$, so we'll have

$$
S'_0(x_0) = k_0\\
S'_i(x_{i+1}) = S'_{i+1}(x_{i+1})=k_{i+1} \qquad 0\le i\le n-1 \\
S'_{n-1}(x_n) = k_n.
$$

The trick then will be to find the $k_i$ that will also give us continuous second derivatives. So we want $S''_i(x_{i+1})=S''_{i+1}(x_{i+1})$ which gives us

$$
2 \frac{a_1-2b_i}{\Delta x_i^2} = 2\frac{b_{i+1}-2a_{i+1}}{\Delta x_{i+1}^2}
$$

Substituting in our expressions for $a_i$ and $b_i$ and simplifying we can write this as

$$
\frac{k_i\Delta x_i-\Delta y_i+2k_{i+1}\Delta x_i -2\Delta y_i}{\Delta x_i^2} = \frac{-k_{i+2}\Delta x_{i+1} + \Delta y_{i+1}-2k_{i+1}\Delta x_{i+1}-2\Delta y_{i+1}}{\Delta x_{i+1}^2}
$$

$$
k_i \frac{1}{\Delta x_i} + 2 k_{i+1} \left(\frac{1}{\Delta x_i} + \frac{1}{\Delta x_{i+1}}\right) +k_{i+2} \frac{1}{\Delta x_{i+1}} = 3\left(\frac{\Delta y_i}{\Delta x_i^2} + \frac{\Delta y_{i+1}}{\Delta x_{i+1}^2}\right)
$$

for $0\le i \le n-2$.

This means we now have a set of $n-1$ linear equations, but we have $n+1$ values of $k_i$.

(Recall $k_i$ is the derivative of our interpolating function at $x_i$, so for every $x_i$ we have a $k_i$).

This means we'll need to come up with two additional constraints before we can solve this system.

### Natural Cubic Splines

To handle this we can impose whatever constraints we like, and this will affect how our spline behaves at the boundaries of our sampled data. The most common approach is what's known as a "natural" cubic spline. This replicates the behaviour of the original mechanical spline which was an elastic rod constrained to pass through a set of "knots" allowing a smooth curve to be drawn through them. The rod is free to take any slope as it passes through the left- and right-most knots, which effectively turns our spline into a straight line right at the boundary points. 

Note: there are several other approaches that could be taken, such as:
- A *Clamped* spline, in which the value of the first derivative at the end points is specified.
- A *Not-a-knot* spline, in which the third derivative of the interpolating function is also required to be continuous at the second and second last point.

In terms of our system of equations, a natural spline means we're setting the second derivative at the boundary points to zero, which gives us the following additional constraints:

$$
S_0''(x_0) = 0 = 2\frac{b_0-2a_0}{\Delta x_0^2} = \frac{2}{\Delta x_0^2}(-k_1\Delta x_0 + \Delta y_0 - 2k_0\Delta x_0 -2\Delta y_0) \\
S_{n-1}''(x_n) = 0 = 2\frac{a_{n-1}-2b_{n-1}}{\Delta x_{n-1}^2} = \frac{2}{\Delta x_{n-1}^2}(k_{n-1}\Delta x_{n-1} + \Delta y_{n-1} + 2k_n\Delta x_{n-1} -2\Delta y_{n-1}).
$$

We can rewrite these as

$$
k_0 \frac{2}{\Delta x_0} + k_1 \frac{1}{\Delta x_0} = 3 \frac{\Delta y_0}{\Delta x_0^2}\\
k_{n-1} \frac{1}{\Delta x_{n-1}} + k_n \frac{2}{\Delta x_{n-1}} = 3 \frac{\Delta y_{n-1}}{\Delta x_{n-1}^2}
$$

And now we have $n+1$ equations for the $n+1$ unknowns $k_i$.

Let's write this set of equations in terms of a matrix equation:
$$\begin{pmatrix} 
\frac{2}{\Delta x_0} & \frac{1}{\Delta x_1} & 0 & 0 & \cdots & 0 \\
\frac{1}{\Delta x_1} & 2(\frac{1}{\Delta x_1} + \frac{1}{\Delta x_2}) & \frac{1}{\Delta x_2} & 0 & \cdots & 0 \\
0 & \frac{1}{\Delta x_2} & 2(\frac{1}{\Delta x_2} + \frac{1}{\Delta x_3}) & \frac{1}{\Delta x_3} & \cdots & 0 \\
\vdots & \ddots & \ddots & \ddots & \ddots & \vdots \\
0 & \cdots & \dots & 0 & \frac{1}{\Delta x_{n-1}} & \frac{2}{\Delta x_{n-1}}
\end{pmatrix}
\begin{pmatrix} k_0 \\ k_1 \\ k_2 \\ \vdots \\ k_n \end{pmatrix} =
\begin{pmatrix}
3\frac{\Delta y_0}{\Delta x_0^2} \\
3(\frac{\Delta y_1}{\Delta x_1^2} + \frac{\Delta y_0}{\Delta x_0^2}) \\
3(\frac{\Delta y_2}{\Delta x_2^2} + \frac{\Delta y_1}{\Delta x_1^2}) \\
\vdots \\
3\frac{\Delta y_{n-1}}{\Delta x_{n-1}^2} \end{pmatrix}
$$

This is a tridiagonal matrix, which we saw how to solve previously. So from here we can write a Python function to perform the natural cubic spline interpolation. The matrices use the differences between consecutive elements which we can obtain from [numpy.ediff1d](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ediff1d.html) or [numpy.diff](https://docs.scipy.org/doc/numpy/reference/generated/numpy.diff.html).

In [ ]:
def spline_interp(xvals, yvals, x):
    '''Natural cubic spline interpolation of the value of y(x) given y(x_i)
    
    Parameters
    ----------
    xvals : float, array
        the x-values we have sampled - they must be in increasing order.
    yvals : float, array
        the value of y at each x we have sampled.
    x : float
        the (x-axis) value at which we want to interpolate.
    
    Returns
    -------
    y : float
        the interpolated value of y(x_interp).
    '''
    
    # First we can find the x and y differences using ediff1d
    dx = np.ediff1d(xvals)
    dy = np.ediff1d(yvals)
    # And let's pre-divide by dx since we usually divide by it.
    invdx = 1.0 / dx
    
    # Now let's generate the packed matrix that we wish to solve
    m_packed = np.empty((3, len(xvals))) # Initialize a 3xN array
    m_packed[0][1:] = invdx
    m_packed[1][0] = 2 * invdx[0]
    m_packed[1][1:-1] = 2 * (invdx[1:] + invdx[:-1])
    m_packed[1][-1] = 2 * invdx[-1]
    m_packed[2][:-1] = invdx

    # And the solution vector
    b = np.empty(len(xvals))
    b[0] = 3 * dy[0] * invdx[0]**2
    b[1:-1] = 3 * (dy[1:]*invdx[1:]**2 + dy[:-1]*invdx[:-1]**2)
    b[-1] = 3 * dy[-1] * invdx[-1]**2

    # Solve Mk=b for a banded matrix
    k = scipy.linalg.solve_banded((1,1), m_packed, b)
    
    # Now we have the ki, let's generate the cubic function in the relevant interval.

    # This will give us the correct value of i for our x_interp value.
    i = np.searchsorted(xvals, x, 'right') - 1
    
    # Let's do this as we laid it out above with
    # $ S_i = (1-t)y_i + ty_{i+1} + t(1-t)[a_i(1-t)+b_i t] $
    # with $t = \frac{x-x_i}{\Delta x_i} $,
    # $ a_i=k_i\Delta x_i-\Delta y_i $ and $ b_i=-k_{i+1}\Delta x_i + \Delta y_i $.
    t = (x - xvals[i]) * invdx[i]
    a = k[i] * dx[i] - dy[i]
    b = -k[i+1] * dx[i] + dy[i]
    
    y_interp = (1-t)*yvals[i] + t*yvals[i+1] + t*(1-t)*(a*(1-t)+b*t)
    return y_interp

In [ ]:
# Note our function has an issue right at the right hand boundary.
# Can you see what this is? How might you fix it?
xplt = np.linspace(0, 9.9, 100)
plt.plot(xplt, spline_interp(x_data, y_data, xplt), 'r-', x_data, y_data, 'bo')
plt.xlabel('x')
plt.ylabel('Spline Interpolated y')
plt.show()

This looks pretty good. It smoothly goes through all the points, without deviating too far from the sampled data anywhere.

Our implementation above has a noticeable inefficiency though. Can you see what it is? 

Hint - Look at the process we go through to generate points for the plot.

How might you improve this?

Let's also plot the approximate first and second derivative to check they are continuous

In [ ]:
# Take dy/dx from our plot points to approximate the derivatives
xplt = np.linspace(0, 9.9, 100)
spline_vals = spline_interp(x_data, y_data, xplt)
dspline = (spline_vals[:-1]-spline_vals[1:])/np.ediff1d(xplt)
plt.plot(xplt[:-1], dspline, 'r-')
plt.xlabel('x')
plt.ylabel('Spline First Derivative')
plt.show()

ddspline = (dspline[:-1]-dspline[1:])/np.ediff1d(xplt)[:-1]
plt.plot(xplt[:-2], ddspline, 'r-')
plt.xlabel('x')
plt.ylabel('Spline Second Derivative')
plt.show()

So we can see both the first and second derivatives seem to be continuous, and it looks like the second derivative goes to zero at the boundaries as we required for the natural spline.

## Doing this with SciPy

SciPy has a range of interpolation functions available in [scipy.interpolate](https://docs.scipy.org/doc/scipy/reference/interpolate.html). There are several functions that will perform similar interpolations but with slightly different interfaces and options. Most will use the FITPACK Fortran library.

Let's first try the [`interp1d`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.interp1d.html) function.

In [ ]:
help(scipy.interpolate.interp1d)

In [ ]:
xvals = np.linspace(0, 10, 1001)
# We use interp1d to generate an interpolating function which will return an interpolated value at any x.
interp_func =  scipy.interpolate.interp1d(x_data, y_data, kind='cubic')
plt.plot(xvals, interp_func(xvals), 'r-', x_data, y_data, 'bo')
plt.xlabel('x')
plt.ylabel('Spline Interpolated y')
plt.show()

If you look carefully however, you'll notice this looks a little different to the spline we generated above. Let's plot both together and see:

In [ ]:
xvals = np.linspace(0, 10, 1001)
# We use interp1d to generate an interpolating function which will return an interpolated value at any x.
interp_func =  scipy.interpolate.interp1d(x_data, y_data, kind='cubic')
plt.plot(xvals, interp_func(xvals), 'r-', xplt, spline_interp(x_data, y_data, xplt), 'g-')
plt.xlabel('x')
plt.ylabel('Spline Interpolated y')
plt.show()

In [ ]:
# Take dy/dx from our plot points to approximate the derivatives
xplt = np.linspace(0, 9.99, 100)
spline_vals = interp_func(xplt)
dspline = (spline_vals[:-1]-spline_vals[1:])/np.ediff1d(xplt)
plt.plot(xplt[:-1], dspline, 'r-')
plt.xlabel('x')
plt.ylabel('Spline First Derivative')
plt.show()

ddspline = (dspline[:-1]-dspline[1:])/np.ediff1d(xplt)[:-1]
plt.plot(xplt[:-2], ddspline, 'r-')
plt.xlabel('x')
plt.ylabel('Spline Second Derivative')
plt.show()

As you can see, this is not a natural cubic spline. It is in fact a *Not-a-knot* spline, which in SciPy is typically the default for cubic splines.

Another SciPy function we could use, which also has options allowing us to pick the type of cubic spline is [scipy.interpolate.CubicSpline](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.CubicSpline.html). The same example as before, using this function gives us

In [ ]:
xvals = np.linspace(0, 10, 1001)
# We use CubicSpline to generate an interpolating function which will return an interpolated value at any x.
# This has the bc_type argument we can use to pick different constraints.
cs_interp =  scipy.interpolate.CubicSpline(x_data, y_data, bc_type='natural')
plt.plot(x_data, y_data, 'bo', label="data")
plt.plot(xvals, cs_interp(xvals), 'r-', label="SciPy")
plt.plot(xplt, spline_interp(x_data, y_data, xplt), 'g--', label="Our Function")
plt.xlabel('x')
plt.ylabel('Spline Interpolated y')
plt.legend()
plt.show()

If you then wanted to find a root from sampled data, you could use the spline interpolator generated by SciPy together with a root finding method discussed previously. However, the [scipy.interpolate.InterpolatedUnivariateSpline](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.InterpolatedUnivariateSpline.html) class can be used to do this more easily. It does not however give you an option for the type of cubic spline to use, and will always generate a "Not-a-knot" spline. It can be used in the same way as the other functions described above. But the object it generates also has several useful methods, such as [roots](https://docs.scipy.org/doc/scipy/reference/generated/scipy.interpolate.InterpolatedUnivariateSpline.roots.html).

In [ ]:
xvals = np.linspace(0, 10, 1001)
# We use UnivariateSpline to generate an interpolating object which has several functions available.

# This has the bc_type argument we can use to pick different constraints.
us_interp =  scipy.interpolate.InterpolatedUnivariateSpline(x_data, y_data)
plt.plot(x_data, y_data, 'bo', label="data")
plt.plot(xvals, us_interp(xvals), 'r-', label="SciPy")
plt.xlabel('x')
plt.ylabel('Spline Interpolated y')
plt.legend()
plt.show()

print("Roots of the interpolation: ", us_interp.roots())